In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

#import json
from google.cloud import bigquery
from datetime import datetime,timedelta

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import TransformerMixin, BaseEstimator

from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split


from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import GradientBoostingClassifier

from imblearn.pipeline import Pipeline

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV,cross_validate

from sklearn.metrics import classification_report


In [2]:
#Localisation du projet name et table_id sur gcp
project="marbotic"

In [3]:
#intégration des credentials 
from google.oauth2 import service_account

key_path = "/Users/antonin/code/AntoninAnq/gcp/marbotic-7d02fac30bd8.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [4]:
def extract(project,credentials):
    # Construct a BigQuery client object.
    client = bigquery.Client(project=project, credentials=credentials)

    query = """
    SELECT
    COUNT(DISTINCT ep.Activity_Name) as unique_activity, 
    sm.session_id,sum(Time_Spent) as Time_Spent,
    any_value(Action_Element_Name) as Action_Element_Name,
    any_value(Profile_Age_days) as Profile_Age_days,
    (SAFE_DIVIDE(SUM(event_type_Activity_Good_Answer) , (SUM(event_type_Activity_Good_Answer)
    + SUM(event_type_Activity_Wrong_Answer))))*100 as good_answer_ratio
    FROM `marbotic.marbotic_dataset.event_properties` as ep
    JOIN `marbotic.marbotic_dataset.sessions_metrics` as sm on ep.session_id = sm.session_id
    GROUP BY sm.session_id
    ORDER BY unique_activity DESC
    """

    query_job = client.query(query)  # Make an API request.
    results=query_job.result().to_dataframe() # Waits for job to complete.

    return pd.DataFrame(results)

In [5]:
#On récupère d'event le nombre d'activity unique par session, le ratio de bonne réponse par sessions et le time spent par sessions
first_data = extract(project,credentials)
first_data

,unique_activity,session_id,Time_Spent,Action_Element_Name,Profile_Age_days,good_answer_ratio
0,42,1.648054e+12,19329.0,Back,1899.0,100.0
1,41,1.646647e+12,21882.0,Back,8434.0,NaN
2,40,1.646297e+12,54083.0,Back,1968.0,100.0
3,39,1.647908e+12,9693.0,Back,2376.0,100.0
4,38,1.646244e+12,16048.0,Back,2227.0,100.0
...,...,...,...,...,...,...
18811,0,1.650445e+12,414.0,None,NaN,NaN
18812,0,1.650582e+12,371.0,None,NaN,NaN
18813,0,1.650516e+12,5286.0,None,NaN,NaN
18814,0,1.650721e+12,2110.0,None,NaN,NaN


In [6]:
def extract_session_metrics(project,credentials):
    # Construct a BigQuery client object.
    client = bigquery.Client(project=project, credentials=credentials)

    query = """
    SELECT *
    FROM `marbotic.marbotic_dataset.sessions_metrics`
    """

    query_job = client.query(query)  # Make an API request.
    results=query_job.result().to_dataframe() # Waits for job to complete.

    return pd.DataFrame(results)

In [7]:
session_metrics_df = extract_session_metrics(project,credentials)
session_metrics_df

,session_id,city,language,os_version,region,country,device_model,start_version,session_end,id,...,event_type_Profile_Update,event_type_Scaffolding_Reset,event_type_Scaffolding_Scaffolding,event_type_Scene_Enter,event_type_Scene_Leave,event_type_Toast_Appear,event_type_Toast_Disappear,session_start,game_events,setup_events
0,1.647421e+12,Certines,French,15.3.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:58:14.214000+00:00,6,...,0.0,0.0,0.0,5.0,1.0,0.0,0.0,2022-03-16 08:56:40.722000+00:00,6.0,0.0
1,1.647421e+12,Certines,French,15.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:58:45.380000+00:00,5,...,0.0,0.0,0.0,4.0,1.0,0.0,0.0,2022-03-16 08:58:28.269000+00:00,5.0,0.0
2,1.648134e+12,Certines,French,15.3.1,Ain,France,"iPad11,6",1.0.1,2022-03-24 15:06:35.838000+00:00,5,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2022-03-24 15:06:21.426000+00:00,5.0,0.0
3,1.647421e+12,Certines,French,15.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:57:59.845000+00:00,5,...,0.0,0.0,0.0,4.0,1.0,0.0,0.0,2022-03-16 08:57:48.381000+00:00,5.0,0.0
4,1.646641e+12,Izernore,French,15.2,Ain,France,iPad 6,1.0.3,2022-03-07 08:25:39.783000+00:00,6,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2022-03-07 08:24:40.937000+00:00,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18811,1.646491e+12,Notting Hill Gate,English,15.1,Royal Kensington and Chelsea,United Kingdom,iPad 6,1.0.7,2022-03-05 14:40:14.517000+00:00,5,...,0.0,0.0,0.0,4.0,1.0,0.0,0.0,2022-03-05 14:39:48.894000+00:00,5.0,0.0
18812,1.652027e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-08 16:39:55.181000+00:00,17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-05-08 16:32:21.326000+00:00,0.0,0.0
18813,1.652029e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-08 17:01:24.501000+00:00,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-05-08 16:58:54.378000+00:00,0.0,0.0
18814,1.652377e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-12 18:05:34.559000+00:00,19,...,0.0,0.0,0.0,10.0,4.0,0.0,0.0,2022-05-12 17:38:47.898000+00:00,19.0,0.0


In [8]:
#on merge les première colonnes crées avec la table sessions metrics
sm_df = session_metrics_df.merge(first_data, on='session_id')

In [9]:
#Calcul du temps de chaque session
sm_df['duration_min'] = (sm_df['session_end']-sm_df['session_start']).apply(lambda x: x.total_seconds()/60)

In [10]:
sm_df

,session_id,city,language,os_version,region,country,device_model,start_version,session_end,id,...,event_type_Toast_Disappear,session_start,game_events,setup_events,unique_activity,Time_Spent,Action_Element_Name,Profile_Age_days,good_answer_ratio,duration_min
0,1.647421e+12,Certines,French,15.3.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:58:14.214000+00:00,6,...,0.0,2022-03-16 08:56:40.722000+00:00,6.0,0.0,0,37.0,None,-1.0,NaN,1.558200
1,1.647421e+12,Certines,French,15.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:58:45.380000+00:00,5,...,0.0,2022-03-16 08:58:28.269000+00:00,5.0,0.0,0,25.0,None,-1.0,NaN,0.285183
2,1.648134e+12,Certines,French,15.3.1,Ain,France,"iPad11,6",1.0.1,2022-03-24 15:06:35.838000+00:00,5,...,0.0,2022-03-24 15:06:21.426000+00:00,5.0,0.0,0,28.0,None,-1.0,NaN,0.240200
3,1.647421e+12,Certines,French,15.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:57:59.845000+00:00,5,...,0.0,2022-03-16 08:57:48.381000+00:00,5.0,0.0,0,20.0,None,-1.0,NaN,0.191067
4,1.646641e+12,Izernore,French,15.2,Ain,France,iPad 6,1.0.3,2022-03-07 08:25:39.783000+00:00,6,...,0.0,2022-03-07 08:24:40.937000+00:00,6.0,0.0,0,858.0,Back,1891.0,NaN,0.980767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18811,1.646491e+12,Notting Hill Gate,English,15.1,Royal Kensington and Chelsea,United Kingdom,iPad 6,1.0.7,2022-03-05 14:40:14.517000+00:00,5,...,0.0,2022-03-05 14:39:48.894000+00:00,5.0,0.0,0,31.0,None,-1.0,NaN,0.427050
18812,1.652027e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-08 16:39:55.181000+00:00,17,...,0.0,2022-05-08 16:32:21.326000+00:00,0.0,0.0,0,2525.0,None,4930.0,NaN,7.564250
18813,1.652029e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-08 17:01:24.501000+00:00,20,...,0.0,2022-05-08 16:58:54.378000+00:00,0.0,0.0,1,834.0,None,4930.0,NaN,2.502050
18814,1.652377e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-12 18:05:34.559000+00:00,19,...,0.0,2022-05-12 17:38:47.898000+00:00,19.0,0.0,1,9393.0,Back,4934.0,100.0,26.777683


In [11]:
def extract_user_metrics(project,credentials):
    # Construct a BigQuery client object.
    client = bigquery.Client(project=project, credentials=credentials)

    query = """
    SELECT user_creation_time, user_id
    FROM `marbotic.marbotic_dataset.users_metrics`
    """

    query_job = client.query(query)  # Make an API request.
    results=query_job.result().to_dataframe() # Waits for job to complete.

    return pd.DataFrame(results)

In [12]:
user_metrics_df = extract_user_metrics(project,credentials)

In [13]:
#on merge la date de création dans le cas ou on veut calculer la maturité du user au moment de la session. Non utilisé dans un premier temps
sm_df = sm_df.merge(user_metrics_df, on='user_id')

In [14]:
sm_df

,session_id,city,language,os_version,region,country,device_model,start_version,session_end,id,...,session_start,game_events,setup_events,unique_activity,Time_Spent,Action_Element_Name,Profile_Age_days,good_answer_ratio,duration_min,user_creation_time
0,1.647421e+12,Certines,French,15.3.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:58:14.214000+00:00,6,...,2022-03-16 08:56:40.722000+00:00,6.0,0.0,0,37.0,None,-1.0,NaN,1.558200,2021-11-20 15:17:18.537000+00:00
1,1.647421e+12,Certines,French,15.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:58:45.380000+00:00,5,...,2022-03-16 08:58:28.269000+00:00,5.0,0.0,0,25.0,None,-1.0,NaN,0.285183,2021-11-20 15:17:18.537000+00:00
2,1.648134e+12,Certines,French,15.3.1,Ain,France,"iPad11,6",1.0.1,2022-03-24 15:06:35.838000+00:00,5,...,2022-03-24 15:06:21.426000+00:00,5.0,0.0,0,28.0,None,-1.0,NaN,0.240200,2021-11-20 15:17:18.537000+00:00
3,1.647421e+12,Certines,French,15.1,Ain,France,"iPad11,6",1.0.1,2022-03-16 08:57:59.845000+00:00,5,...,2022-03-16 08:57:48.381000+00:00,5.0,0.0,0,20.0,None,-1.0,NaN,0.191067,2021-11-20 15:17:18.537000+00:00
4,1.646641e+12,Izernore,French,15.2,Ain,France,iPad 6,1.0.3,2022-03-07 08:25:39.783000+00:00,6,...,2022-03-07 08:24:40.937000+00:00,6.0,0.0,0,858.0,Back,1891.0,NaN,0.980767,2021-12-07 18:08:20.505000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18811,1.649501e+12,Kensington,English,15.1,Royal Kensington and Chelsea,United Kingdom,"iPad12,1",1.0.5,2022-04-09 10:45:26.022000+00:00,1,...,2022-04-09 10:45:26.022000+00:00,1.0,0.0,1,3573.0,None,NaN,NaN,0.000000,2021-12-19 14:16:39.549000+00:00
18812,1.652027e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-08 16:39:55.181000+00:00,17,...,2022-05-08 16:32:21.326000+00:00,0.0,0.0,0,2525.0,None,4930.0,NaN,7.564250,2022-05-08 16:29:14.356000+00:00
18813,1.652029e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-08 17:01:24.501000+00:00,20,...,2022-05-08 16:58:54.378000+00:00,0.0,0.0,1,834.0,None,4930.0,NaN,2.502050,2022-05-08 16:29:14.356000+00:00
18814,1.652377e+12,Hamburg,English,14.6,Free and Hanseatic City of Hamburg,Germany,iPad Pro,4.1.3,2022-05-12 18:05:34.559000+00:00,19,...,2022-05-12 17:38:47.898000+00:00,19.0,0.0,1,9393.0,Back,4934.0,100.0,26.777683,2022-05-08 16:29:14.356000+00:00


In [15]:
sm_df.columns

Index(['session_id', 'city', 'language', 'os_version', 'region', 'country',
       'device_model', 'start_version', 'session_end', 'id', 'user_id',
       'event_type_Account_Sign_In', 'event_type_Account_Sign_Out',
       'event_type_Account_Sign_Up', 'event_type_Account_Update_Request',
       'event_type_Account_Update_Success', 'event_type_Action_Click',
       'event_type_Action_ConversionClickRedeem',
       'event_type_Action_ConversionClickSubscribe',
       'event_type_Activation_Game', 'event_type_Activation_New_Activation',
       'event_type_Activation_Piece', 'event_type_Activation_Product',
       'event_type_Activity_Good_Answer', 'event_type_Activity_Start',
       'event_type_Activity_Stop', 'event_type_Activity_Wrong_Answer',
       'event_type_Error_Error', 'event_type_Game_Pause',
       'event_type_Game_Start', 'event_type_Game_Stop',
       'event_type_OB_ClickStore', 'event_type_OB_ClickTrial',
       'event_type_OB_CommPreview', 'event_type_OB_DiscoverTrial',
  

In [16]:
#On calcul le nombre de sessions avant celle en cours
def nb_session(user_id,session_start):
    return sm_df[(sm_df['user_id']==user_id) & (sm_df['session_start']<session_start)].shape[0]

In [17]:
sm_df['previous_session'] = sm_df.apply(lambda x:nb_session(x.user_id,x.session_start), axis=1)

In [18]:
#On calcul le temps depuis la dernière sessions
def last_session_time(user_id,session_start):
    last_session = sm_df[(sm_df['user_id']==user_id) & (sm_df['session_start']<session_start)]\
    .sort_values(by='session_start',ascending=True)\
    .groupby('user_id').last()['session_start']
    if len(last_session.values)==0:
        return timedelta(0)
    else:
        last_session = last_session.dt.to_pydatetime()[0].replace(tzinfo=None)
        return (session_start.to_pydatetime().replace(tzinfo=None) - last_session)

In [19]:
#(sm_df[(sm_df['user_id']==65616.0) & (sm_df['session_start']<'2022-03-16 08:58:28.269000+00:00')].sort_values(by='session_start',ascending=True).groupby('user_id').last()['session_start'].dt.to_pydatetime()[0].replace(tzinfo=None))

In [20]:
sm_df['time_since_last_session'] = sm_df.apply(lambda x:last_session_time(x.user_id,x.session_start), axis=1)

In [21]:
#Vérification pour un utilisateur donné, le début de toutes ses sessions ordonné et l'écart avec la dernière session
sm_df[sm_df['user_id']==66337.0].sort_values(by='session_start',ascending=True)[['session_start','time_since_last_session']]

,session_start,time_since_last_session
10,2022-01-12 15:34:18.911000+00:00,0 days 00:00:00
11,2022-03-05 16:00:00.822000+00:00,52 days 00:25:41.911000
9,2022-03-05 20:23:34.583000+00:00,0 days 04:23:33.761000
6,2022-03-07 07:15:12.051000+00:00,1 days 10:51:37.468000
7,2022-03-07 07:52:31.465000+00:00,0 days 00:37:19.414000
4,2022-03-07 08:24:40.937000+00:00,0 days 00:32:09.472000
5,2022-05-09 09:50:43.882000+00:00,63 days 01:26:02.945000
8,2022-05-10 06:57:04.671000+00:00,0 days 21:06:20.789000
12,2022-05-10 14:45:39.525000+00:00,0 days 07:48:34.854000
16,2022-05-11 06:42:18.649000+00:00,0 days 15:56:39.124000


In [22]:
#sm_df.groupby('user_id').agg({'time_since_last_session':'mean','setup_events':sum,'game_events':sum,'id':'count'})['id'].value_counts()

In [23]:
#Durée moyenne entre deux sessions
sm_df[sm_df['time_since_last_session']!=timedelta(0)]['time_since_last_session'].mean()

Timedelta('3 days 04:41:50.486608937')

In [24]:
#Conversion de la colonne time_since_last_session timedelta en timestampe[min]
sm_df['time_since_last_session_min']=sm_df['time_since_last_session'].map(lambda x : x.total_seconds()/60)

**Ajout de la maturité au moment du début de la sessions**

In [25]:
sm_df["week_maturity"] =(sm_df["session_start"]-sm_df["user_creation_time"]).map(lambda x: int(np.ceil(x.days/7))+1)

**Ajout du type d'utilisateur depuis la table user_augmented**

In [26]:
def extract_user_type(project,credentials):
    # Construct a BigQuery client object.
    client = bigquery.Client(project=project, credentials=credentials)

    query = """
    SELECT Type, user_id
    FROM `marbotic.marbotic_dataset.user_augmented`
    """

    query_job = client.query(query)  # Make an API request.
    results=query_job.result().to_dataframe() # Waits for job to complete.

    return pd.DataFrame(results)

In [27]:
user_type_df = extract_user_type(project,credentials)
user_type_df

BadRequest: 400 Unrecognized name: Type at [2:12]

Location: US
Job ID: ce0b7f85-c960-475e-bfd1-235e44458aa4


In [ ]:
user_type_df.drop_duplicates(subset='user_id',inplace=True)

In [ ]:
sm_df = sm_df.merge(user_type_df, on='user_id')

In [ ]:
#Durée moyenne entre deux sessions pour les parents
sm_df[sm_df['Type']=='Parent'][sm_df['time_since_last_session']!=timedelta(0)]['time_since_last_session'].mean()

In [ ]:
week_maturity = sm_df['week_maturity']
min_last_session = sm_df['time_since_last_session_min']

In [ ]:
sm_df['session_start_day'] = sm_df['session_start'].apply(lambda x: x.weekday()+1)

In [ ]:
sm_df['session_start_month'] = sm_df['session_start'].apply(lambda x: x.month)

In [ ]:
#Gros pic de session au mois de mars (= Merge de l'appli ? puis uniquement décroissance)
sm_df[sm_df['Type']=='Parent'][['session_start_month']].hist();

In [ ]:
#On ne retrouve pas la saisonalité hebdomadaire annoncée
sm_df[sm_df['Type']=='Parent'][['session_start_day']].hist();

In [ ]:
plt.figure(figsize=(15, 10))

sns.scatterplot(x=week_maturity,y=min_last_session,alpha=0.4,hue=sm_df['Type'])

plt.axhline(30240, ls='--', linewidth=1.5, color='red')
plt.text(92,30240, "3 semaines depuis la dernière session")

plt.axhline(21160, ls='--', linewidth=1.5, color='red')
plt.text(92,21160, "2 semaines depuis la dernière session")

plt.axhline(10080, ls='--', linewidth=1.5, color='red')
plt.text(92,10080, "1 semaines depuis la dernière session")

plt.show()

In [ ]:
plt.figure(figsize=(15, 10))

sns.scatterplot(x=week_maturity,y=sm_df['good_answer_ratio'],alpha=0.4,hue=sm_df['Type'])

#plt.axhline(30240, ls='--', linewidth=1.5, color='red')
#plt.text(92,30240, "3 semaines depuis la dernière session")

#plt.axhline(21160, ls='--', linewidth=1.5, color='red')
#plt.text(92,21160, "2 semaines depuis la dernière session")

#plt.axhline(10080, ls='--', linewidth=1.5, color='red')
#plt.text(92,10080, "1 semaines depuis la dernière session")
#plt.ylim(0,100)
plt.show()

In [ ]:
#On drope la colone time_since_last_session en timedelta car incompatbile BQ
sm_df.drop('time_since_last_session',axis=1,inplace=True)

## Correlation

In [ ]:
# Heatmap
corr = sm_df.corr()
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns,
        cmap= "YlGnBu");

In [ ]:
corr_df = corr.unstack().reset_index() # Unstack correlation matrix 
corr_df.columns = ['feature_1','feature_2', 'correlation'] # rename columns
corr_df.sort_values(by="correlation",ascending=False, inplace=True) # sort by correlation
corr_df = corr_df[corr_df['feature_1'] != corr_df['feature_2']] # Remove self correlation
corr_df.head(20)

In [ ]:
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]

In [ ]:
to_drop

## Labelling

In [ ]:
#On considère le seuil de churn à 3 semaine.
#On est en situation de churn si la session suivante arrive dans plus de 3 semaines.
CHURN_THRESHOLD = timedelta(days=7)

#On utilise une constante TODAY pour evaluer le risque de churn des dernières session.
#Dans notre cas TODAY est le max de session end. On considère que l'extract s'arrète au moment de la dernière session de jeu dont on dispose
TODAY = datetime(2022, 5, 13, 0, 0)

In [ ]:
#sm_df[(sm_df['user_id']==65616.0)& ('2022-03-16 08:58:14.214000+00:00'<sm_df['session_end'])].sort_values(by='session_end',ascending=True).groupby('user_id').first()['session_start']

In [ ]:
#On calcul le temps jusqu'à la prochaine sessions si superieur au seuil de churn. Label_risque de churn = 1
def labelling_churn(user_id,session_end,session_start,CHURN_THRESHOLD,today):
    next_session = sm_df[(sm_df['user_id']==user_id) & (session_end<sm_df['session_end'])]\
    .sort_values(by='session_end',ascending=True)\
    .groupby('user_id').first()['session_start']
    if len(next_session.values) == 0:
        if today - session_start.to_pydatetime().replace(tzinfo=None)> CHURN_THRESHOLD :
            return 1
        else:
            return 0
    else:
        delta = (next_session - session_end)
        if delta.iloc[0] < CHURN_THRESHOLD:
            return 0
        else:
            return 1

In [ ]:
sm_df['churn_risk'] = sm_df.apply(lambda x:labelling_churn(x.user_id,x.session_end,x.session_start,CHURN_THRESHOLD,TODAY), axis=1)

In [ ]:
#Vérification du label de churn sur l'utilisateur déja vérifié plus haut.
sm_df[sm_df['user_id']==66337.0].sort_values(by='session_start',ascending=True)[['session_start','time_since_last_session_min','churn_risk']]

In [ ]:
sm_df[sm_df['Type']=='Parent']['churn_risk'].value_counts()

In [ ]:
(1901/(13090+1901))*100

In [ ]:
sm_df[sm_df['Type']=='Teacher']['churn_risk'].value_counts()

In [ ]:
(329/(329+3495))*100

## Cleaning

In [ ]:
#No duplicates
sm_df.duplicated().sum()

In [ ]:
((sm_df.isnull().sum().sort_values(ascending=False)/len(sm_df))*100).head(10)

good_answer_ratio Nan correspond to session with no event type answer. On remplace les Nan par -1

In [ ]:
answer_ratio_imputer = SimpleImputer(strategy='constant',fill_value=-1)
sm_df['good_answer_ratio'] = answer_ratio_imputer.fit_transform(sm_df[['good_answer_ratio']])

In [ ]:
#34% de Nan, on drop cette colonne tant qu'on n'a pas d'information supplémentaire sur la signification des Nan
sm_df['Action_Element_Name'].value_counts()

In [ ]:
sm_df.drop('Action_Element_Name',axis=1,inplace=True)

In [ ]:
#Pour Profile_Age_days(17%), Time_Spent(0.03) on remplace par la médiane car un énorme outlier(-2911184.0) et 3341 valeur = -1
med_imputer = SimpleImputer(strategy='median')
sm_df['Profile_Age_days'] = med_imputer.fit_transform(sm_df[['Profile_Age_days']])
sm_df['Time_Spent'] = med_imputer.fit_transform(sm_df[['Time_Spent']])

In [ ]:
#Pour Type, il manque une valeur on remplace par most frequent
mmost_freq_imputer = SimpleImputer(missing_values=None,strategy='most_frequent')
sm_df['Type'] = mmost_freq_imputer.fit_transform(sm_df[['Type']])

In [ ]:
((sm_df.isnull().sum().sort_values(ascending=False)/len(sm_df))*100).head(10)

**Traitement des outliers extremes**

In [ ]:
# A controle Time_spent max, Profile_Age_days min
#On supprime la session avec le time_spent = 17162486.0 de user_id = 23370.0 
#Il a 70 sessions avec une médiane de Time_spent 7571
sm_df[['Time_Spent']].max()

In [ ]:
sm_df.shape

In [ ]:
sm_df.drop(axis=0,index=sm_df[sm_df['Time_Spent']==17162486.0].index,inplace=True)

In [ ]:
#On supprime 2 autres lignes avec Time_Spent > 1 000 000 Cela concerne le même utilsateur qui a 20 sessions
sm_df.drop(axis=0,index=sm_df[sm_df['Time_Spent']==2190329.0].index,inplace=True)
sm_df.drop(axis=0,index=sm_df[sm_df['Time_Spent']==1093473.0].index,inplace=True)

In [ ]:
# On supprime l'utilisateur avec un age énorme négatif. Session unique user_creation_time : 27-22-2021
sm_df.drop(axis=0,index=sm_df[sm_df['Profile_Age_days']==-2911184.0].index,inplace=True)

## processing des colonnes

In [ ]:
sm_df.select_dtypes(include='object')

In [ ]:
class CustomOHE(OneHotEncoder):
    def fit(self, *args, **kwargs):
        return super().fit(*args, **kwargs) 
    
    def transform(self,*args, **kwargs):
        return pd.DataFrame(super().transform(*args, **kwargs), columns=self.get_feature_names_out())

    def fit_transform(self,*args, **kwargs):
        return pd.DataFrame(super().fit_transform(*args, **kwargs), columns=self.get_feature_names_out())

In [ ]:
cat_transformer = CustomOHE(sparse = False, handle_unknown = "ignore", drop='if_binary')
cat_selection = sm_df[['Type']].columns

In [ ]:
class CustomMinMaxScale(MinMaxScaler):
    def fit(self, *args, **kwargs):
        return super().fit(*args, **kwargs) 
    
    def transform(self,*args, **kwargs):
        return pd.DataFrame(super().transform(*args, **kwargs), columns=self.get_feature_names_out())

    def fit_transform(self,*args, **kwargs):
        return pd.DataFrame(super().fit_transform(*args, **kwargs), columns=self.get_feature_names_out())

In [ ]:
#On retire des éléments très corrélés
num_transformer = CustomMinMaxScale()
num_selection = sm_df.select_dtypes(include=['float64','int64']).drop(['churn_risk','session_id','id','user_id','event_type_Toast_Disappear','event_type_OB_ProductHelp','event_type_OB_ProfileName'],axis=1).columns

In [ ]:
class Cyclical_month_Encoder(BaseEstimator, TransformerMixin):
    '''Custom transformer to convert month into cyclical encoding cos and sin'''
    def _init_(self):
        pass 
    
    def fit(self, X,y=None):
        return super().fit(*args, **kwargs) 
    
    def transform(self, X,y=None):
        X = pd.DataFrame(X)
        df = pd.DataFrame()
        cst = (2*np.pi/12)
        df['month_cos'] = X.apply(lambda x: np.cos(cst*x))
        df['month_sin'] = X.apply(lambda x: np.sin(cst*x))
        return df
    
    def fit_transform(self, X,y=None):
        X = pd.DataFrame(X)
        df = pd.DataFrame()
        cst = (2*np.pi/12)
        df['month_cos'] = X.apply(lambda x: np.cos(cst*x))
        df['month_sin'] = X.apply(lambda x: np.sin(cst*x))
        return df
    
    def get_feature_names_out(self,X):
        return 'month_cos','month_sin'

In [ ]:
month_transformer = Cyclical_month_Encoder()
month_selection = sm_df[['session_start_month']].columns

In [ ]:
class Cyclical_day_Encoder(BaseEstimator, TransformerMixin):
    '''Custom transformer to convert day into cyclical encoding cos and sin'''
    def _init_(self):
        pass 
    
    def fit(self, X,y=None):
        return self
    
    def transform(self, X,y=None):
        X = pd.DataFrame(X)
        df = pd.DataFrame()
        cst = (2*np.pi/7)
        df['day_cos'] = X.apply(lambda x: np.cos(cst*x))
        df['day_sin'] = X.apply(lambda x: np.sin(cst*x))
        return df
    
    def get_feature_names_out(self,X):
        return 'day_cos','day_sin'

In [ ]:
day_transformer = Cyclical_day_Encoder()
day_selection = sm_df[['session_start_day']].columns

In [ ]:
class CustomColumnTransformer(ColumnTransformer):
    
    def fit(self, *args, **kwargs):
        return super().fit(*args, **kwargs)
    
    def transform(self, X, *args, **kwargs):
        return pd.DataFrame(super().transform(X, *args, **kwargs), columns=self.get_feature_names_out())
    
    def fit_transform(self, X, *args, **kwargs):
        return pd.DataFrame(super().fit_transform(X, *args, **kwargs), columns=self.get_feature_names_out())

In [ ]:
sm_df.shape

In [ ]:
len(num_selection) + len(cat_selection) + len(month_selection) + len(day_selection)

In [ ]:
preprocessor = CustomColumnTransformer([
    ('num_tr',num_transformer,num_selection),
    ('cat_tr',cat_transformer,cat_selection ),
    ('month_tr',month_transformer,month_selection ),
    ('day_tr',day_transformer,day_selection )],
    remainder = 'drop'
)

In [ ]:
preprocessor

In [ ]:
#oversample = SMOTE()
# la bonne pratique et d'oversampler et de downsampler
#over = SMOTE(sampling_strategy=0.1)
#under = RandomUnderSampler(sampling_strategy=0.5)

In [ ]:
#X_smote = preprocessor.fit_transform(sm_df)

In [ ]:
#y_smote = sm_df['churn_risk'].copy()

In [ ]:
#X_smote, y_smote = oversample.fit_resample(X_smote, y_smote)


In [ ]:
sm_df['churn_risk'].value_counts()

## Preparation du jeu de donnée

**On downsample les sessions sans churn risque pour en avoir autant que de churn risque**

In [ ]:
#def downsample_churn_risk(df):
#    sm_df_churn_risque = df[df['churn_risk']==1]
#    sm_df_no_churn_risque = df[df['churn_risk']==0]
#    sm_df_no_churn_risque = sm_df_no_churn_risque.loc\
#    [np.random.choice(sm_df_no_churn_risque.index,size = sm_df_churn_risque.shape[0])]
#    
#    sm_df_balanced = pd.concat([sm_df_no_churn_risque,sm_df_churn_risque],axis=0)
#    X= sm_df_balanced.drop('churn_risk',axis=1).copy()
#    y = sm_df_balanced['churn_risk'].copy()
#    return X,y

In [ ]:
#X,y = downsample_churn_risk(sm_df)

**On va plutôt oversample avec SMOT directement dans le pipeline** Il faut uniquement des données numériques

In [ ]:
X = sm_df.drop('churn_risk',axis=1).copy()
y = sm_df['churn_risk'].copy()

In [ ]:
X.shape,y.shape

In [ ]:
y.value_counts()

On sépare le jeu de donnée en train et test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

## Feature selection

In [ ]:
X_feature_sel = pd.DataFrame(preprocessor.fit_transform(X_train))

In [ ]:
#Permet d'obtenir l'importance des features au regard des autres
#pour chaque modèle on l'entraine 10 fois en mélangeant les valeurs d'une colonne gardant toutes les autres identiques
 
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression().fit(X_feature_sel, y_train) # Fit model
# on peut mettre un keyworld arg max_iter=1000, par défaut le modèle a une limite (100, à vérifier)

permutation_score = permutation_importance(log_model, X_feature_sel, y_train, n_repeats=10) # Perform Permutation

importance_df = pd.DataFrame(np.vstack((X_feature_sel.columns,
                                        permutation_score.importances_mean)).T) # Unstack results
importance_df.columns=['feature','score decrease']

importance_df.sort_values(by="score decrease", ascending = False) # Order by importance

**La méthode de feature permutation ne donne rien de pertinent**

## Model

In [ ]:
class CustomSelectPercentile(SelectPercentile):
    def fit(self, *args, **kwargs):
        return super().fit(*args, **kwargs) 
    
    def transform(self,*args, **kwargs):
        return pd.DataFrame(super().transform(*args, **kwargs), columns=self.get_feature_names_out())

    def fit_transform(self,*args, **kwargs):
        return pd.DataFrame(super().fit_transform(*args, **kwargs), columns=self.get_feature_names_out())

In [ ]:
pipe = make_pipeline(preprocessor,CustomSelectPercentile(percentile=75), RandomForestClassifier())

In [ ]:
pipe

In [ ]:
smote_pipe = Pipeline([('prep',preprocessor),\
                       ('smote',SMOTE()),\
                       ('feat_selec',CustomSelectPercentile(percentile=50)),\
                       ('RF',RandomForestClassifier())])

In [ ]:
smote_pipe.get_params()

In [ ]:
grid_search = GridSearchCV(
    smote_pipe, 
    param_grid={
        'RF__n_estimators': [400,450],
        'RF__max_depth': [40],
        'RF__min_samples_split' : [15],
        'RF__max_samples' : [0.8],
        'RF__n_jobs' : [-1]
    },
    cv=5,
    scoring="accuracy",
verbose=1)

In [ ]:
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
best_search = grid_search.best_estimator_

In [ ]:
#Pipeline de rééquilibrage du test set
test_set_smote = Pipeline([('prep',preprocessor),\
                       ('smote',SMOTE())])

In [ ]:
X_test, y_test = test_set_smote.fit_resample(X_test,y_test)

In [ ]:
#Predicting with oversample X_test
y_pred = best_search.predict(X_test)

In [ ]:
target_names = ['classe 0','Classe 1']
print(classification_report(y_test,y_pred,target_names=target_names))

In [ ]:
ran_for_C = RandomForestClassifier(max_depth=40,max_samples= 0.8,min_samples_split= 15, n_estimators=450)
log_reg_C = LogisticRegression(max_iter = 1000,n_jobs=-1, C= 1.5,solver= 'newton-cg')
svc_C = SVC(C= 10,gamma= 1,kernel= 'rbf')
knn_C = KNeighborsClassifier(n_jobs= -1,n_neighbors= 30,p= 1,weights= 'distance')
ada_b_C = AdaBoostClassifier(algorithm= 'SAMME.R',learning_rate= 1,n_estimators= 50)
ridge_C = RidgeClassifier(alpha= 2,max_iter= 1000,solver= 'lsqr')

ensemble = StackingClassifier(
    estimators = [("rf", ran_for_C),
                  ("knn", knn_C),
                  ('log',log_reg_C),
                  ('svc',svc_C),
                  ('ada',ada_b_C),
                  ('ridge',ridge_C)
                 ],
    final_estimator = GradientBoostingClassifier())
stacked_pipe = Pipeline([('prep',preprocessor),\
                       ('smote',SMOTE()),\
                       ('feat_selec',CustomSelectPercentile(percentile=50)),\
                       ('ensemble_stack',ensemble)])

In [ ]:
cv_results_stacked = cross_validate(stacked_pipe, X_train, y_train, scoring='accuracy', cv=5)


In [ ]:
score_stacked = cv_results_stacked['test_score'].mean()
score_stacked

In [ ]:
stacked_pipe.fit(X_train, y_train)

In [ ]:
y_pred_stacked = stacked_pipe.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred_stacked,target_names=target_names))

In [ ]:
#Confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
label = ['No churn risk', 'Churn risk'] 
cm = confusion_matrix(y_test, y_pred_stacked)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=label)
disp.plot();

In [ ]:
stacked_pipe.named_steps.keys()

In [ ]:
#Analyse de la variance des colonnes gardées
pd.DataFrame(stacked_pipe.named_steps["customselectpercentile"].scores_,index=stacked_pipe.named_steps["customselectpercentile"].feature_names_in_).sort_values(by=0,ascending=False)[:33]

**Quels sont les colonnes qui ont été gardée dans le modèle de stacking**

In [ ]:
columns_selected_pipe = make_pipeline(preprocessor,CustomSelectPercentile(percentile=50))
columns_selected = columns_selected_pipe.fit_transform(X_train, y_train).columns

In [ ]:
columns_selected = [f[8:] for f in columns_selected]

In [ ]:
#33 colonnes gardées
columns_selected

## Modèle uniquement sur les parents

In [ ]:
parent_df = sm_df[sm_df['Type']=="Parent"]

In [ ]:
X_parent, y_parent = downsample_churn_risk(parent_df)

In [ ]:
X_train_parent, X_test_parent, y_train_parent, y_test_parent = train_test_split(X_parent, y_parent, test_size=0.30)

In [ ]:
stacked_pipe.fit(X_train_parent, y_train_parent)

In [ ]:
y_pred_parent_stacked = stacked_pipe.predict(X_test_parent)

In [ ]:
print(classification_report(y_test_parent,y_pred_parent_stacked,target_names=target_names))

## Upload dans BQ

**Upload de la table en cours dans BQ**

In [ ]:
###Localisation du projet name et table_id sur gcp
#project="marbotic"
#table_id = "marbotic_dataset.session_augmented"
#client = bigquery.Client(project,credentials)
#
#job = client.load_table_from_dataframe(sm_df, table_id) 
#
#job.result()  # Wait for the job to complete.
#table = client.get_table(table_id)  # Make an API request.
#print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id))

In [ ]:
9000/(50)

In [ ]:
180/2